In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.5 MB/s eta 0:00:00


In [ ]:
from torch_geometric.nn import GATConv

In [ ]:
import numpy as np
import pickle
import os
from pathlib import Path

class DEAPDataLoader:


    def __init__(self, data_path):

        self.data_path = Path(data_path)

    def load_subject_data(self, subject_id):

        filename = self.data_path / f's{subject_id:02d}.dat'

        if not filename.exists():
            raise FileNotFoundError(f"File not found: {filename}")

        with open(filename, 'rb') as f:
            subject = pickle.load(f, encoding='latin1')

        return subject['data'], subject['labels']

    def load_all_subjects(self, n_subjects=32):

        all_data = []
        all_labels = []
        subject_ids = []

        print(f"Loading DEAP data for {n_subjects} subjects...")

        for subject_id in range(1, n_subjects + 1):
            try:
                data, labels = self.load_subject_data(subject_id)
                all_data.append(data)
                all_labels.append(labels)
                subject_ids.extend([subject_id] * len(data))

                if subject_id % 10 == 0:
                    print(f"Loaded {subject_id}/{n_subjects} subjects")

            except FileNotFoundError:
                print(f"Warning: Subject {subject_id} data not found, skipping...")
                continue

        all_data = np.vstack(all_data)
        all_labels = np.vstack(all_labels)
        subject_ids = np.array(subject_ids)

        print(f"Total loaded: {len(all_data)} trials")
        print(f"Data shape: {all_data.shape}")
        print(f"Labels shape: {all_labels.shape}")

        return all_data, all_labels, subject_ids

    def preprocess_for_model(self, data, labels, use_32_channels=True):

        if use_32_channels:
            X = data[:, :32, :]
        else:
            X = data

        X = X[:, :, :7680]


        valence = labels[:, 0]
        arousal = labels[:, 1]


        y_valence = (valence > 5).astype(np.int32)
        y_arousal = (arousal > 5).astype(np.int32)

        print(f"\nPreprocessed data shape: {X.shape}")
        print(f"Valence distribution: Low={np.sum(y_valence==0)}, High={np.sum(y_valence==1)}")
        print(f"Arousal distribution: Low={np.sum(y_arousal==0)}, High={np.sum(y_arousal==1)}")

        return X, y_valence, y_arousal

    def split_train_test(self, X, y_valence, y_arousal, subject_ids=None,
                        test_size=0.2, random_state=42):

        from sklearn.model_selection import train_test_split

        if subject_ids is None:

            indices = np.arange(len(X))
            train_idx, test_idx = train_test_split(
                indices, test_size=test_size, random_state=random_state,
                stratify=y_valence
            )
        else:

            unique_subjects = np.unique(subject_ids)
            n_test_subjects = int(len(unique_subjects) * test_size)

            np.random.seed(random_state)
            test_subjects = np.random.choice(
                unique_subjects, n_test_subjects, replace=False
            )

            test_idx = np.where(np.isin(subject_ids, test_subjects))[0]
            train_idx = np.where(~np.isin(subject_ids, test_subjects))[0]

        X_train = X[train_idx]
        X_test = X[test_idx]
        y_valence_train = y_valence[train_idx]
        y_valence_test = y_valence[test_idx]
        y_arousal_train = y_arousal[train_idx]
        y_arousal_test = y_arousal[test_idx]

        print(f"\nTrain set: {len(X_train)} trials")
        print(f"Test set: {len(X_test)} trials")

        return (X_train, X_test, y_valence_train, y_valence_test,
                y_arousal_train, y_arousal_test)



def main():

    DEAP_PATH = "/content/drive/My Drive/deap-dataset/data_preprocessed_python"


    if not os.path.exists(DEAP_PATH):
        print(f"ERROR: DEAP data path not found: {DEAP_PATH}")
        print("\nPlease:")
        print("1. Download DEAP dataset from: http://www.eecs.qmul.ac.uk/mmv/datasets/deap/")
        print("2. Extract the 'data_preprocessed_python' folder")
        print("3. Update DEAP_PATH variable above with the correct path")
        return


    loader = DEAPDataLoader(DEAP_PATH)


    all_data, all_labels, subject_ids = loader.load_all_subjects(n_subjects=32)


    X, y_valence, y_arousal = loader.preprocess_for_model(all_data, all_labels)

    X_train, X_test, y_val_train, y_val_test, y_ar_train, y_ar_test = \
        loader.split_train_test(X, y_valence, y_arousal, test_size=0.2)


    print("\n" + "="*60)
    print("DATA READY FOR TRAINING!")
    print("="*60)
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")


    print("\nTo train the model, use:")
    print("from emotion_recognition import EEGEmotionRecognition")
    print("model = EEGEmotionRecognition(n_channels=32)")
    print("model.train(X_train, y_val_train, y_ar_train, augment_factor=1.0)")
    print("valence_pred, arousal_pred = model.predict(X_test)")

    return X_train, X_test, y_val_train, y_val_test, y_ar_train, y_ar_test


if __name__ == "__main__":

    data = main()


    if data is not None:
        X_train, X_test, y_val_train, y_val_test, y_ar_train, y_ar_test = data

        print("\n" + "="*60)
        print("Next step: Train the model (this takes a while!)")
        print("="*60)



Loading DEAP data for 32 subjects...
Loaded 10/32 subjects
Loaded 20/32 subjects
Loaded 30/32 subjects
Total loaded: 1280 trials
Data shape: (1280, 40, 8064)
Labels shape: (1280, 4)

Preprocessed data shape: (1280, 32, 7680)
Valence distribution: Low=1011, High=269
Arousal distribution: Low=982, High=298

Train set: 1024 trials
Test set: 256 trials

DATA READY FOR TRAINING!
X_train shape: (1024, 32, 7680)
X_test shape: (256, 32, 7680)

To train the model, use:
from emotion_recognition import EEGEmotionRecognition
model = EEGEmotionRecognition(n_channels=32)
model.train(X_train, y_val_train, y_ar_train, augment_factor=1.0)
valence_pred, arousal_pred = model.predict(X_test)

Next step: Train the model (this takes a while!)


In [ ]:
from collections import Counter
print(Counter(y_val_train))
print(Counter(y_ar_train))

Counter({np.int32(0): 809, np.int32(1): 215})
Counter({np.int32(0): 785, np.int32(1): 239})


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"


class ChannelEncoder(nn.Module):
    def __init__(self, n_channels=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_channels, 64, kernel_size=7, stride=4, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=7, stride=4, padding=3)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 32, kernel_size=7, stride=4, padding=3)
        self.bn3 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return x

class ContrastiveProjector(nn.Module):
    def __init__(self, in_ch=32, proj_dim=128):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_ch, proj_dim),
            nn.ReLU(),
            nn.Linear(proj_dim, proj_dim)
        )

    def forward(self, feat_map):
        pooled = feat_map.mean(dim=2)
        proj = self.fc(pooled)
        proj = F.normalize(proj, dim=1)
        return proj

class ContrastiveLearning(nn.Module):
    def __init__(self, n_channels=32, proj_dim=128):
        super().__init__()
        self.encoder = ChannelEncoder(n_channels)
        self.projector = ContrastiveProjector(in_ch=32, proj_dim=proj_dim)

    def forward(self, x):
        feat_map = self.encoder(x)
        proj = self.projector(feat_map)
        return feat_map, proj

def nt_xent_loss(z1, z2, temperature=0.1):
    assert z1.shape == z2.shape
    B = z1.shape[0]
    z = torch.cat([z1, z2], dim=0)
    sim = torch.matmul(z, z.T) / temperature
    labels = torch.arange(B, device=z.device)
    labels = torch.cat([labels + B, labels])


    large_neg = -1e9
    sim_masked = sim.clone()
    idx = torch.arange(2*B, device=z.device)
    sim_masked[idx, idx] = large_neg

    loss = F.cross_entropy(sim_masked, labels)
    return loss


def eeg_augment(x, noise_std=0.05, shift_max=500, scale_range=(0.8,1.2), dropout_prob=0.1):
    B, C, T = x.shape
    out = x.clone()


    out = out + torch.randn_like(out) * noise_std


    shifts = torch.randint(-shift_max, shift_max+1, (B,))
    for i, s in enumerate(shifts):
        out[i] = torch.roll(out[i], shifts=s.item(), dims=1)


    scales = torch.rand(B,1,1, device=x.device) * (scale_range[1]-scale_range[0]) + scale_range[0]
    out = out * scales


    mask = torch.rand(B, C, 1, device=x.device) > dropout_prob
    out = out * mask.float()

    return out


X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32).to(device)

cl_model = ContrastiveLearning(n_channels=32, proj_dim=128).to(device)
optimizer_cl = torch.optim.Adam(cl_model.parameters(), lr=3e-3)
batch_size = 64
epochs_cl = 120

train_dataset = TensorDataset(X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

cl_model.train()
for epoch in range(epochs_cl):
    total_loss = 0.0
    for (x_batch,) in train_loader:
        x_batch = x_batch.to(device)
        x1 = eeg_augment(x_batch)
        x2 = eeg_augment(x_batch)

        _, z1 = cl_model(x1)
        _, z2 = cl_model(x2)

        loss = nt_xent_loss(z1, z2, temperature=0.1)
        optimizer_cl.zero_grad()
        loss.backward()
        optimizer_cl.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"[CL] Epoch {epoch+1}/{epochs_cl}, Loss: {avg_loss:.4f}")

cl_model.eval()
with torch.no_grad():
    train_feat_maps = cl_model.encoder(X_train_tensor)
    test_feat_maps  = cl_model.encoder(X_test_tensor)

train_features = train_feat_maps.cpu().numpy()
test_features  = test_feat_maps.cpu().numpy()

[CL] Epoch 1/120, Loss: 2.2712
[CL] Epoch 2/120, Loss: 1.3786
[CL] Epoch 3/120, Loss: 1.2237
[CL] Epoch 4/120, Loss: 1.1220
[CL] Epoch 5/120, Loss: 1.0838
[CL] Epoch 6/120, Loss: 0.9536
[CL] Epoch 7/120, Loss: 0.9132
[CL] Epoch 8/120, Loss: 0.7925
[CL] Epoch 9/120, Loss: 0.8169
[CL] Epoch 10/120, Loss: 0.7442
[CL] Epoch 11/120, Loss: 0.7333
[CL] Epoch 12/120, Loss: 0.6599
[CL] Epoch 13/120, Loss: 0.6281
[CL] Epoch 14/120, Loss: 0.6077
[CL] Epoch 15/120, Loss: 0.6140
[CL] Epoch 16/120, Loss: 0.5285
[CL] Epoch 17/120, Loss: 0.5122
[CL] Epoch 18/120, Loss: 0.5241
[CL] Epoch 19/120, Loss: 0.5012
[CL] Epoch 20/120, Loss: 0.4627
[CL] Epoch 21/120, Loss: 0.4531
[CL] Epoch 22/120, Loss: 0.4526
[CL] Epoch 23/120, Loss: 0.4205
[CL] Epoch 24/120, Loss: 0.4283
[CL] Epoch 25/120, Loss: 0.3965
[CL] Epoch 26/120, Loss: 0.4204
[CL] Epoch 27/120, Loss: 0.3671
[CL] Epoch 28/120, Loss: 0.4118
[CL] Epoch 29/120, Loss: 0.4012
[CL] Epoch 30/120, Loss: 0.3458
[CL] Epoch 31/120, Loss: 0.3324
[CL] Epoch 32/120

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from collections import Counter

device = "cuda" if torch.cuda.is_available() else "cpu"

class Generator(nn.Module):
    def __init__(self, code_dim=256, n_classes=2, out_channels=32, out_time=120, hidden_dim=1024):
        super().__init__()

        self.label_emb = nn.Embedding(n_classes, code_dim)

        self.net = nn.Sequential(
            nn.Linear(code_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(True),
            nn.Linear(hidden_dim, out_channels * out_time),
            nn.Tanh()
        )
        self.out_channels = out_channels
        self.out_time = out_time

    def forward(self, z, labels):

        label_embed = self.label_emb(labels)
        x = z + label_embed
        out = self.net(x)
        return out.view(-1, self.out_channels, self.out_time)


class Discriminator(nn.Module):
    def __init__(self, in_channels=32, in_time=120, n_classes=2):
        super().__init__()
        self.conv = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv1d(in_channels, 64, kernel_size=3, padding=1)),
            nn.LeakyReLU(0.2),
            nn.utils.spectral_norm(nn.Conv1d(64, 128, kernel_size=3, padding=1)),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.fc(self.conv(x))


def train_gan_for_class(real_samples, class_label,
                        code_dim=256, n_classes=2,
                        batch_size=64, epochs=150, d_steps=5, lr=2e-4):
    """Train a class-conditioned GAN for one emotion class."""
    real_tensor = torch.tensor(real_samples, dtype=torch.float32)
    loader = DataLoader(TensorDataset(real_tensor), batch_size=batch_size, shuffle=True, drop_last=True)

    G = Generator(code_dim, n_classes).to(device)
    D = Discriminator(n_classes=n_classes).to(device)

    opt_G = torch.optim.Adam(G.parameters(), lr=lr * 1.5, betas=(0.5, 0.9))
    opt_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.9))
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        d_loss_total, g_loss_total = 0, 0
        for (x_real,) in loader:
            x_real = x_real.to(device)
            B = x_real.size(0)
            y_real = torch.ones((B, 1), device=device)
            y_fake = torch.zeros((B, 1), device=device)

            for _ in range(d_steps):
                z = torch.randn(B, code_dim, device=device)
                labels = torch.full((B,), class_label, dtype=torch.long, device=device)
                fake = G(z, labels).detach()

                opt_D.zero_grad()
                loss_real = criterion(D(x_real), y_real)
                loss_fake = criterion(D(fake), y_fake)
                loss_d = (loss_real + loss_fake) / 2
                loss_d.backward()
                opt_D.step()
                d_loss_total += loss_d.item()


            z = torch.randn(B, code_dim, device=device)
            labels = torch.full((B,), class_label, dtype=torch.long, device=device)
            opt_G.zero_grad()
            fake = G(z, labels)
            loss_g = criterion(D(fake), y_real)
            loss_g.backward()
            opt_G.step()
            g_loss_total += loss_g.item()

        if (epoch + 1) % 10 == 0:
            print(f"[Class {class_label}] Epoch {epoch+1}/{epochs} | D_loss={d_loss_total/len(loader):.4f} | G_loss={g_loss_total/len(loader):.4f}")

    return G


def generate_synthetic_for_class(G, n_samples, class_label, code_dim=256):
    G.eval()
    synth = []
    with torch.no_grad():
        bs = min(256, n_samples)
        for _ in range(int(np.ceil(n_samples / bs))):
            z = torch.randn(bs, code_dim, device=device)
            labels = torch.full((bs,), class_label, dtype=torch.long, device=device)
            x_fake = G(z, labels).cpu().numpy()
            synth.append(x_fake)
    synth = np.vstack(synth)[:n_samples]
    return synth


def classwise_gan_augment(X_train, y_train, balance_mode='equalize', code_dim=256, epochs=150):
    counts = Counter(y_train)
    majority = max(counts.values())

    X_synth_all, y_synth_all = [], []
    for c in np.unique(y_train):
        real_c = X_train[y_train == c]
        if balance_mode == 'equalize':
            target = majority - len(real_c)
            if c == 1:
                target = int(target * 1.25)
        else:
            target = len(real_c)
        if target <= 0:
            continue
        print(f"\n Training GAN for class {c}: real={len(real_c)}, synth_target={target}")
        G = train_gan_for_class(real_c, class_label=c, code_dim=code_dim, epochs=epochs)
        synth = generate_synthetic_for_class(G, target, class_label=c, code_dim=code_dim)
        X_synth_all.append(synth)
        y_synth_all.append(np.full(target, c, dtype=np.int32))

    if X_synth_all:
        X_aug = np.vstack([X_train] + X_synth_all)
        y_aug = np.concatenate([y_train] + y_synth_all)
    else:
        X_aug, y_aug = X_train, y_train

    print("\n Class counts after augmentation:", Counter(y_aug))
    return X_aug, y_aug


if __name__ == "__main__":
    print("\n--- GAN augmentation on CL embeddings ---")


    X_train_feat = train_features
    y_train_val = y_val_train
    y_train_ar = y_ar_train

    print("\n Training GANs for Valence balancing...")
    X_val_aug, y_val_aug = classwise_gan_augment(X_train_feat, y_train_val,
                                                 balance_mode='equalize',
                                                 code_dim=256, epochs=200)

    print("\n Training GANs for Arousal balancing...")
    X_ar_aug, y_ar_aug = classwise_gan_augment(X_train_feat, y_train_ar,
                                               balance_mode='equalize',
                                               code_dim=256, epochs=150)

    np.savez("deap_gan_augmented_features.npz",
             X_val=X_val_aug, y_val=y_val_aug,
             X_ar=X_ar_aug, y_ar=y_ar_aug)

    print("\n GAN augmentation finished successfully!")


--- GAN augmentation on CL embeddings ---

 Training GANs for Valence balancing...

 Training GAN for class 1: real=215, synth_target=742
[Class 1] Epoch 10/200 | D_loss=3.2521 | G_loss=0.7521
[Class 1] Epoch 20/200 | D_loss=2.9715 | G_loss=0.9386
[Class 1] Epoch 30/200 | D_loss=2.6705 | G_loss=0.9649
[Class 1] Epoch 40/200 | D_loss=2.5637 | G_loss=0.9777
[Class 1] Epoch 50/200 | D_loss=2.3246 | G_loss=1.1548
[Class 1] Epoch 60/200 | D_loss=2.3094 | G_loss=1.1169
[Class 1] Epoch 70/200 | D_loss=2.0439 | G_loss=1.3121
[Class 1] Epoch 80/200 | D_loss=1.8800 | G_loss=1.4742
[Class 1] Epoch 90/200 | D_loss=1.8341 | G_loss=1.3981
[Class 1] Epoch 100/200 | D_loss=1.6604 | G_loss=1.5027
[Class 1] Epoch 110/200 | D_loss=1.5670 | G_loss=1.3777
[Class 1] Epoch 120/200 | D_loss=1.4187 | G_loss=1.6530
[Class 1] Epoch 130/200 | D_loss=1.3088 | G_loss=1.7584
[Class 1] Epoch 140/200 | D_loss=1.2435 | G_loss=1.8853
[Class 1] Epoch 150/200 | D_loss=1.1051 | G_loss=2.0150
[Class 1] Epoch 160/200 | D_lo

In [ ]:
from collections import Counter
print(Counter(y_val_aug))
print(Counter(y_ar_aug))

Counter({np.int32(1): 957, np.int32(0): 809})
Counter({np.int32(1): 921, np.int32(0): 785})


In [ ]:

class GNN_GAT_EmotionClassifier(nn.Module):
    def __init__(self, n_channels=32, feat_length=None, hidden_sizes=[50,28,7,2],
                 dropout_rates=[0.22,0.0,0.0,0.05], target_node=15, gat_heads=4):
        super().__init__()
        self.n_channels = n_channels
        self.feat_length = feat_length
        self.target_node = target_node
        self.gat_heads = gat_heads


        A_init = initialize_adjacency_matrix(n_channels)
        self.A = nn.Parameter(A_init.to(device))


        self.layers = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        in_features = feat_length
        for hidden_size, dropout_rate in zip(hidden_sizes[:-1], dropout_rates[:-1]):
            self.layers.append(nn.Linear(in_features, hidden_size))
            self.dropouts.append(nn.Dropout(dropout_rate) if dropout_rate>0 else nn.Identity())
            in_features = hidden_size

        final_dim = hidden_sizes[-1]
        gat_out_channels = max(1, final_dim // gat_heads)

        self.gat = GATConv(
            in_channels=hidden_sizes[-2],
            out_channels=gat_out_channels,
            heads=gat_heads,
            concat=True,
            dropout=dropout_rates[-1]
        )


        gat_output_dim = gat_heads * gat_out_channels
        if gat_output_dim != final_dim:
            self.final_proj = nn.Linear(gat_output_dim, final_dim)
        else:
            self.final_proj = None

    def forward(self, S):
        B, C, feat_dim = S.shape
        X = S


        for layer, dropout in zip(self.layers, self.dropouts):
            A_hat = torch.softmax(self.A, dim=1)
            X = torch.einsum('ij,bjk->bik', A_hat, X)

            B, C, feat_dim = X.shape
            X_flat = X.reshape(B*C, feat_dim)
            X_flat = torch.relu(layer(X_flat))
            X_flat = dropout(X_flat)
            X = X_flat.reshape(B, C, -1)


        A_hat = torch.softmax(self.A, dim=1)
        edge_index = (A_hat > 1e-5).nonzero(as_tuple=False).t().contiguous().to(device)

        X_gat_list = []
        for b in range(B):
            X_b = X[b]
            out_b = self.gat(X_b, edge_index)
            X_gat_list.append(out_b)

        X_gat = torch.stack(X_gat_list, dim=0)


        if self.final_proj is not None:
            X_gat = self.final_proj(X_gat)


        output = X_gat[:, self.target_node, :]
        return output



import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight


try:
    from torch_geometric.nn import GATConv
except ImportError:
    print("Warning: torch_geometric not found. Please install it.")

device = "cuda" if torch.cuda.is_available() else "cpu"


def initialize_adjacency_matrix(n_channels=32, lambda_threshold=5.0, theta=2.0):
    """Initialize adjacency matrix using Gaussian kernel."""
    coords_3d = np.array([
        [0, 6, 8], [2, 6, 8], [-2, 6, 8], [0, 4, 8],
        [-3, 4, 6], [3, 4, 6], [-1, 2, 7], [1, 2, 7],
        [-2, 1, 6], [2, 1, 6], [-3, 0, 4], [3, 0, 4],
        [-1, -1, 5], [1, -1, 5], [0, -2, 4], [0, -3, 3],
        [-2, -3, 3], [2, -3, 3], [-3, -4, 2], [3, -4, 2],
        [-2, 3, 6], [2, 3, 6], [-3, 1, 5], [3, 1, 5],
        [-2, -1, 4], [2, -1, 4], [-1, -3, 3], [1, -3, 3],
        [0, -1, 5], [0, 1, 7], [-1, 0, 6], [1, 0, 6]
    ])
    W = np.zeros((n_channels, n_channels))

    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                W[i, j] = 0
            else:
                dist = euclidean(coords_3d[i], coords_3d[j])
                if dist <= lambda_threshold:
                    W[i, j] = np.exp(-(dist**2) / (2 * theta**2))
                else:
                    W[i, j] = 0
    return torch.tensor(W, dtype=torch.float32)



def train_gnn(model, X_train, y_train,
              batch_size=100, epochs=400, lr=5e-5,
              weight_decay=1e-4):


    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)
    num_classes = len(le.classes_)

    print(f"Original labels: {np.unique(y_train)}")
    print(f"Encoded labels: {np.unique(y_train_encoded)}")
    print(f"Number of classes: {num_classes}")

    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train_encoded, dtype=torch.long)
    )


    labels = y_train_encoded
    classes = np.unique(labels)
    class_sample_count = np.array([len(np.where(labels == t)[0]) for t in classes])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in labels])

    samples_weight = torch.from_numpy(samples_weight).double()
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)


    from sklearn.utils.class_weight import compute_class_weight
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train_encoded)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

    print(f"Class weights shape: {class_weights.shape}")
    print(f"Class weights: {class_weights}")

    criterion = nn.CrossEntropyLoss(weight=class_weights)

    train_losses, train_accs = [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds, train_labels = [], []

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(outputs.argmax(dim=1).cpu().numpy())
            train_labels.extend(y_batch.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, train_preds)

        train_losses.append(train_loss)
        train_accs.append(train_acc)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")

    return {
        'train_losses': train_losses,
        'train_accs': train_accs,
        'label_encoder': le
    }



def evaluate_gnn(model, X_test, y_test, label_encoder=None, batch_size=100):

    if label_encoder is not None:
        y_test_encoded = label_encoder.transform(y_test)
    else:
        y_test_encoded = y_test

    test_dataset = TensorDataset(
        torch.tensor(X_test, dtype=torch.float32),
        torch.tensor(y_test_encoded, dtype=torch.long)
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model.eval()
    test_preds, test_labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            test_preds.extend(outputs.argmax(dim=1).cpu().numpy())
            test_labels.extend(y_batch.numpy())

    test_acc = accuracy_score(test_labels, test_preds)
    print("\n" + "="*50)
    print(f" Test Accuracy: {test_acc*100:.2f}%")
    print("="*50)

    return {'test_acc': test_acc, 'predictions': test_preds, 'labels': test_labels}


def run_gnn_pipeline(X_train, y_train, X_test, y_test, target_node=15):

    num_classes = len(np.unique(y_train))

    print(f"\nDetected {num_classes} unique classes in training data")
    print(f"Unique train labels: {np.unique(y_train)}")
    print(f"Unique test labels: {np.unique(y_test)}")


    model = GNN_GAT_EmotionClassifier(
        n_channels=X_train.shape[1],
        feat_length=X_train.shape[2],
        hidden_sizes=[50, 28, 7, num_classes],
        dropout_rates=[0.22, 0.0, 0.0, 0.05],
        target_node=target_node,
        gat_heads=4
    ).to(device)

    print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
    print(f"Model output dimension: {num_classes}")


    with torch.no_grad():
        test_input = torch.randn(1, X_train.shape[1], X_train.shape[2]).to(device)
        test_output = model(test_input)
        print(f"ACTUAL model output shape: {test_output.shape}")
        print(f"Expected output shape: torch.Size([1, {num_classes}])")


    history = train_gnn(model, X_train, y_train)


    results = evaluate_gnn(model, X_test, y_test, label_encoder=history['label_encoder'])

    return model, history, results

In [ ]:
if __name__ == "__main__":

    print("\n Training GNN for Valence...")
    model_valence, val_history, val_results = run_gnn_pipeline(
    X_val_aug,
    y_val_aug,
    test_features,
    y_val_test
    )
    val_pred = val_results['predictions']


    print("\n Training GNN for Arousal...")
    model_arousal, ar_history, ar_results = run_gnn_pipeline(
    X_ar_aug,
    y_ar_aug,
    test_features,
    y_ar_test
    )
    ar_pred = ar_results['predictions']


 Training GNN for Valence...

Detected 2 unique classes in training data
Unique train labels: [0 1]
Unique test labels: [0 1]
Model initialized with 8755 parameters
Model output dimension: 2
ACTUAL model output shape: torch.Size([1, 2])
Expected output shape: torch.Size([1, 2])
Original labels: [0 1]
Encoded labels: [0 1]
Number of classes: 2
Class weights shape: torch.Size([2])
Class weights: tensor([1.0915, 0.9227], device='cuda:0')
Epoch 10/400 | Train Loss: 0.6787, Acc: 0.5294
Epoch 20/400 | Train Loss: 0.6556, Acc: 0.5176
Epoch 30/400 | Train Loss: 0.5988, Acc: 0.6925
Epoch 40/400 | Train Loss: 0.4972, Acc: 0.8851
Epoch 50/400 | Train Loss: 0.4061, Acc: 0.8817
Epoch 60/400 | Train Loss: 0.3566, Acc: 0.8851
Epoch 70/400 | Train Loss: 0.3413, Acc: 0.8834
Epoch 80/400 | Train Loss: 0.3012, Acc: 0.8975
Epoch 90/400 | Train Loss: 0.3015, Acc: 0.8958
Epoch 100/400 | Train Loss: 0.3073, Acc: 0.8879
Epoch 110/400 | Train Loss: 0.3033, Acc: 0.8907
Epoch 120/400 | Train Loss: 0.2887, Acc: 